In [10]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config
from deeppavlov.core.commands.infer import *
import sys
sys.path.insert(0, '../../')
from model.pipeline.text_normalizer import *
from model.pipeline.embedder import *
from model.pipeline.CNN_model import *
import pandas as pd
from sklearn.metrics import accuracy_score,f1_score, classification_report

In [11]:
import os
os.getcwd()

'/home/lsm/projects/CF/CF_question_classifier/bot_deeppavlov/intents/subs/pay'

In [16]:
config = read_json('pay_config_temp.json')
model = build_model_from_config(config)
def eval_ipavlov(in_x):
    in_s = []
    in_s.append('{}::'.format(in_x))
    return model(in_s)[0][0]

Instructions for updating:
keep_dims is deprecated, use keepdims instead


2018-06-24 17:20:58.349 WARNING in 'tensorflow'['tf_logging'] at line 126: From /home/lsm/anaconda3/envs/ipavlov_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1264: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


2018-06-24 17:20:58.403 WARNING in 'tensorflow'['tf_logging'] at line 126: From /home/lsm/anaconda3/envs/ipavlov_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2885: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


2018-06-24 17:20:58.427 WARNING in 'tensorflow'['tf_logging'] at line 126: From /home/lsm/anaconda3/envs/ipavlov_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1349: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [17]:
def predict_ipavlov(in_text):
    rp = model.pipe[0][-1]([in_text])
    for i in range(1,len(model.pipe)-1):
        rp = model.pipe[i][-1](rp)
    res = model.pipe[-1][-1](rp, predict_proba = True)
    dec = proba2labels(res, confident_threshold=model.pipe[-1][-1].confident_threshold, classes=model.pipe[-1][-1].classes)[0][0]
    return {
        'decision': dec,
        'confidence': np.max(res)
           }

In [18]:
rabbit = 'Я только про оплату не совсем поняла . Если вам на карту , то доставка ****** . А если на почте , то только * % от суммы или еще + ****** ? )'
eval_ipavlov(rabbit)

'Способы оплаты'

In [11]:
while True:
    mes = input()
    res = predict_ipavlov(mes)
    print('decision: %s\nconfidence: %1.4f'%(res['decision'], res['confidence']))

Здравствуйте . Мой заказ приняли , когда отправлять .
decision: Доставка
confidence: 0.7059
Добрый вечер , посылочка пришла , а почта не работает . завтра заберу , жду не дождусь
decision: Оплата
confidence: 0.5170
Хорошо , я еще на работе отчеты
decision: Общеразговорные
confidence: 0.5502
Можно с правой стороны наверху не большой кармашек , для ******** . спасибо и извините .
decision: Характеристики продукта
confidence: 0.9484
Здравствуйте. Хочу заказать костюм
decision: Доставка
confidence: 0.4476
Здравствуйте. Хочу купить залат
decision: Общеразговорные
confidence: 0.2689
Сколько стоит халат
decision: Характеристики продукта
confidence: 0.8120
Привет. Хочу штаны
decision: Характеристики продукта
confidence: 0.5453


KeyboardInterrupt: 

In [19]:
df_test = pd.read_csv('df_test.csv')

In [20]:
preds = df_test.comment.apply(eval_ipavlov)

In [21]:
accuracy_score(preds, df_test.label)

0.7205882352941176

In [22]:
f1_score(preds, df_test.label, average = 'weighted')

0.7341093156979857

In [23]:
f1_score(preds, df_test.label, average = 'micro')

0.7205882352941176

In [117]:
#classification_report(preds, df_test.reason)

In [6]:
comments_total = pd.read_excel('../Цветоформа tree.xlsx',sheet_name='Лист2').iloc[:,:2]

In [7]:
comments_total['decision'] = comments_total.comment.apply(lambda x: predict_ipavlov(x)['decision'])
comments_total['confidence'] = comments_total.comment.apply(lambda x: predict_ipavlov(x)['confidence'])

In [9]:
comments_total.to_excel('cf_predictions.xlsx')